In [ ]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers[torch] torch datasets accelerate scikit-learn

import pandas as pd
import numpy as np
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


In [ ]:
# %% 2. การเตรียมข้อมูลและตัวอย่าง Few-Shot

print("กำลังโหลด Thai Fake News Detection Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face Hub
raw_dataset = load_dataset("EXt1/Thai-True-Fake-News")

In [ ]:
from datasets import Dataset, DatasetDict

# แปลง Dataset เป็น DataFrame ก่อน
df = raw_dataset['train'].to_pandas()

# เปลี่ยนชื่อคอลัมน์ใน DataFrame
df = df.rename(columns={'Title': 'text', 'Verification_Status': 'label'})

# อัปเดต raw_dataset โดยแปลงจาก DataFrame กลับเป็น Dataset

# สร้าง Dataset ใหม่จาก DataFrame ที่เปลี่ยนชื่อคอลัมน์แล้ว
updated_dataset = Dataset.from_pandas(df)

# สร้าง DatasetDict ใหม่
raw_dataset = DatasetDict({
    'train': updated_dataset
})

print("เปลี่ยนชื่อคอลัมน์สำเร็จ:")
print(f"DataFrame columns: {df.columns.tolist()}")
print(f"Dataset features: {raw_dataset['train'].features}")
print("\nตัวอย่างข้อมูลหลังเปลี่ยนชื่อ:")
print(df[['text', 'label']].head())

In [ ]:

df = raw_dataset['train'].to_pandas()
df = df.dropna(subset=['text', 'label']).drop_duplicates(subset=['text'])

# จำลองคอลัมน์จากโจทย์
df['news_title'] = df['text'].str[:30]
df['news_content'] = df['text'].str[30:]

# --- คัดเลือกตัวอย่างคุณภาพสูงสำหรับ Few-Shot Prompt ---
# ตัวอย่างที่ 1: ข่าวจริง (Real News)
real_news_example = df[df['label'] == 'real'].iloc[10] # เลือกตัวอย่างที่ชัดเจน
# ตัวอย่างที่ 2: ข่าวปลอม (Fake News)
fake_news_example = df[df['label'] == 'fake'].iloc[5] # เลือกตัวอย่างที่ชัดเจน

# --- สร้างข้อมูลสำหรับ Test Set เพื่อจำลองการแข่งขัน ---
test_df = df.sample(n=100, random_state=42) # ลดขนาดเพื่อความรวดเร็วในการทดลอง
test_df['news_id'] = range(len(test_df))
test_df_labels = test_df['label'].apply(lambda x: 1 if x == 'fake' else 0).values

print("คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ")
print("\n--- ตัวอย่างข่าวจริง ---")
print(f"Title: {real_news_example['news_title']}")
print("\n--- ตัวอย่างข่าวปลอม ---")
print(f"Title: {fake_news_example['news_title']}")


In [ ]:
# %% 3. การโหลดโมเดล LLM และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
print(f"กำลังโหลด Tokenizer และโมเดล LLM: '{model_id}'...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อลดการใช้ VRAM และเพิ่มความเร็ว
    device_map="auto"           # โหลดโมเดลข้าม GPU อัตโนมัติหากมีหลายตัว
)
model_llm.eval() # ตั้งเป็น evaluation mode

print("โหลดโมเดล LLM สำเร็จ")


In [ ]:
# %% 4. การออกแบบ Prompt (Few-Shot & Chain-of-Thought)

def create_few_shot_prompt(news_title, news_content):
    """
    สร้าง Prompt ที่สมบูรณ์แบบ Few-shot และ Chain-of-Thought
    """
    # นี่คือ Chat Template ที่เราจะส่งให้โมเดล
    chat = [
        # 1. System Message: กำหนดบทบาทและคำสั่งหลัก
        {"role": "system", "content": "คุณคือนักสืบสวนข่าวปลอมผู้เชี่ยวชาญด้านการเงิน คุณต้องวิเคราะห์ข่าวตามขั้นตอนที่กำหนด และตอบกลับเป็น JSON object ที่มี key 'analysis' และ 'prediction' เท่านั้น โดยค่าของ prediction ต้องเป็น 'เชื่อถือได้' หรือ 'ข้อมูลบิดเบือน' อย่างใดอย่างหนึ่ง"},

        # 2. Few-shot Example 1 (ข่าวจริง)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{real_news_example['news_title']}"
เนื้อหา: "{real_news_example['news_content']}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
        {"role": "assistant", "content": """{
    "analysis": "1. ข่าวมีการอ้างอิงถึงหน่วยงานภาครัฐและข้อมูลที่ตรวจสอบได้ 2. ภาษาที่ใช้เป็นทางการและเน้นข้อเท็จจริง ไม่มีการชี้นำอารมณ์ 3. ไม่มีการกล่าวอ้างเกินจริงหรือรับประกันผลตอบแทน 4. สรุปแล้ว ข่าวนี้มีความน่าเชื่อถือสูง",
    "prediction": "ข่าวจริง"
}"""},

        # 3. Few-shot Example 2 (ข่าวปลอม)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{fake_news_example['news_title']}"
เนื้อหา: "{fake_news_example['news_content']}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
        {"role": "assistant", "content": """{
    "analysis": "1. ข่าวไม่มีการอ้างอิงแหล่งข้อมูลที่ชัดเจน และมาจากบุคคลที่ไม่เป็นที่รู้จัก 2. ภาษาที่ใช้เร่งเร้าและกระตุ้นความโลภ ('ด่วน', 'รวยเร็ว') มากกว่าให้ข้อมูล 3. มีการกล่าวอ้างถึงผลตอบแทนที่สูงเกินจริงและง่ายดาย 4. สรุปแล้ว ข่าวนี้มีลักษณะของข้อมูลบิดเบือนเพื่อหลอกลวง",
    "prediction": "ข่าวปลอม"
}"""},

        # 4. User Query (ข่าวที่เราต้องการวิเคราะห์)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{news_title}"
เนื้อหา: "{news_content}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
    ]
    return chat

# ทดสอบสร้าง Prompt
test_prompt = create_few_shot_prompt("ทดสอบหัวข้อ", "ทดสอบเนื้อหา")
print("สร้าง Template Prompt สำเร็จ")
# print(json.dumps(test_prompt, indent=2, ensure_ascii=False)) # สามารถ uncomment เพื่อดูโครงสร้างเต็มๆ ได้


In [ ]:
# %% 5. การทำนายผลด้วย LLM และการ Parsing

results = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting with LLM"):
    # สร้าง Prompt สำหรับข่าวแต่ละชิ้น
    prompt = create_few_shot_prompt(row['news_title'], row['news_content'])
    
    # แปลง Prompt เป็น Token IDs และส่งเข้า GPU
    inputs = tokenizer.apply_chat_template(
        prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)

    # จำกัดความยาว Input เพื่อป้องกัน Error
    if inputs.shape[1] > 4096:
        inputs = inputs[:, -4096:]

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model_llm.generate(
            inputs,
            max_new_tokens=256,         # จำกัดความยาวของคำตอบ
            do_sample=False,            # ไม่ต้องสุ่มคำตอบเพื่อให้ผลลัพธ์คงที่
            pad_token_id=tokenizer.eos_token_id
        )
    
    # ถอดรหัสคำตอบ
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # พยายาม Parse JSON จากคำตอบ
    try:
        start_index = response_text.find('{')
        end_index = response_text.rfind('}')
        json_part = response_text[start_index : end_index + 1]
        data = json.loads(json_part)
        prediction = data.get('prediction', 'error')
    except (json.JSONDecodeError, AttributeError, ValueError):
        prediction = 'error' # หากเกิดข้อผิดพลาดในการ parse

    results.append(prediction)

# เพิ่มผลลัพธ์ลงใน DataFrame
test_df['llm_prediction'] = results

print("\nทำนายผลด้วย LLM สำเร็จ")
print("ตัวอย่างผลการทำนาย:")
print(test_df[['news_title', 'label', 'llm_prediction']].head(10))


In [ ]:
# %% 6. การประเมินผลและสร้างไฟล์ Submission

# แปลง prediction ที่เป็น string เป็น label ตัวเลข (0 หรือ 1)
def to_label(pred_text):
    if "ข้อมูลบิดเบือน" in pred_text:
        return 1
    elif "เชื่อถือได้" in pred_text:
        return 0
    else:
        return -1 # คืนค่า -1 หากเป็น Error หรือคำตอบไม่ตรงรูปแบบ

test_df['predicted_label'] = test_df['llm_prediction'].apply(to_label)

# กรองแถวที่เกิด Error ออกก่อนประเมินผล
eval_df = test_df[test_df['predicted_label'] != -1]
actual_labels = eval_df['label'].apply(lambda x: 1 if x == 'fake' else 0)
predicted_labels = eval_df['predicted_label']


# ประเมินผลเทียบกับคำตอบจริง (เพื่อดูประสิทธิภาพของ Prompt)
if not eval_df.empty:
    accuracy = accuracy_score(actual_labels, predicted_labels)
    f1 = f1_score(actual_labels, predicted_labels)
    print("\n--- ประสิทธิภาพของโมเดลบน Test Set ---")
    print(f"accuracy: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"จำนวนที่ทำนายได้สำเร็จ: {len(eval_df)}/{len(test_df)}")
else:
    print("\nไม่สามารถประเมินผลได้เนื่องจาก LLM ไม่ได้ให้คำตอบที่ถูกต้อง")

# --- สร้างไฟล์ Submission ---
# สำหรับแถวที่เกิด Error อาจจะต้องใช้วิธีสำรอง หรือเติมค่าที่พบบ่อยที่สุด (Majority class)
# ในที่นี้ เราจะเติม 0 สำหรับแถวที่เกิด error
final_predictions = test_df['predicted_label'].copy()
final_predictions[final_predictions == -1] = 0 # เติม 0 (เชื่อถือได้) สำหรับแถวที่ error

submission_df = pd.DataFrame({
    'news_id': test_df['news_id'],
    'label': final_predictions.astype(int)
})

print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission_llm.csv", index=False)

print("\nสร้างไฟล์ submission_llm.csv สำเร็จ!")
